## Pré-Processamento - Classificação de Sequências em Texto com LSTMs

In [1]:
# Imports
import os
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import SpatialDropout1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import Dropout
from keras.layers.convolutional import Convolution1D
from keras.layers.convolutional import MaxPooling1D
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
from time import time
import re
import unidecode
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from plotly.offline import iplot
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')
from os.path import isfile
from keras.callbacks import EarlyStopping
from keras.models import load_model

In [2]:
parent_dir = 'C:/Programas/DSA/NLP/Ocorrencia/Data'
subject_dirs = [os.path.join(parent_dir, dir) for dir in os.listdir(parent_dir) if os.path.isdir(os.path.join(parent_dir, dir))]
subject_dirs = [w.replace('\\', '/') for w in subject_dirs]

filelist = []
for dir in subject_dirs:
    csv_files = [os.path.join(dir, csv) for csv in os.listdir(dir) if os.path.isfile(os.path.join(dir, csv)) and csv.endswith('.csv')]
    for file in csv_files:
        filelist.append(file)
filelist = [w.replace('\\', '/') for w in filelist]
filelist

['C:/Programas/DSA/NLP/Ocorrencia/Data/B01129/B01129-Lesao.csv',
 'C:/Programas/DSA/NLP/Ocorrencia/Data/C01155/C01155-Furto.csv',
 'C:/Programas/DSA/NLP/Ocorrencia/Data/C01157/C01157-Roubo.csv',
 'C:/Programas/DSA/NLP/Ocorrencia/Data/C01171/C01171-Estelionato.csv',
 'C:/Programas/DSA/NLP/Ocorrencia/Data/T00008/T00008-Acidente-Sem.csv',
 'C:/Programas/DSA/NLP/Ocorrencia/Data/T00009/T00009-Acidente-Com.csv']

In [7]:
df = []
data = []

In [9]:
data = pd.DataFrame()
columns = range(0,4)
for c, f in enumerate(filelist) :
    key = "file%i" % c
#    frame = pd.read_csv( (f), skiprows = 0, index_col=0, names=columns ) 
    frame = pd.read_csv( (f), skiprows = 1, names=columns )     
    frame["key"] = key
    data = data.append(frame,ignore_index=True)
df = data.rename(columns = {2: 'des_historico', 0: 'codigo'}, inplace = False)
df = df[['des_historico', 'codigo']]
df

,des_historico,codigo
0,DURANTE PATRULHAMENTO FOMOS ACIONADOS PELA VIT...,B01129
1,"DESLOCAMOS ATE A RUA JOSE LINDOLFO LEITE, Nº 4...",B01129
2,ACIONADOS COMPARECEMOS NO LOCAL SUPRA CITADO O...,B01129
3,EM PATRULHAMENTO PELO LOCAL A GUARNICAO DEPARO...,B01129
4,"SEGUNDO A VITIMA, APOS DESENTENDIMENTO FAMILIA...",B01129
...,...,...
1739995,FOMOS PROCURADOS PELO SENHOR CLENON O QUAL NOS...,T00009
1739996,"EMPENHADOS PELO COPOM, COMPARECEMOS AO LOCAL (...",T00009
1739997,"ACIONADOS PELO COPOM, DESLOCAMOS PARA O CRUZAM...",T00009
1739998,"EM PATRULHAMENTO, DEPARAMOS COM A VP 26995 COM...",T00009


In [10]:
#filtro = ["C01155", "T00008", "C01157", "T00009", "B01129", "C01171"]
#df = df[df.codigo.isin(filtro)]
#df = df.reset_index(drop=True)
#print(df)

In [11]:
reg19 = list(df[df.index == 19]['des_historico'])
reg19

["DURANTE REALIZACAO DE POLICIAMENTO NO LOCAL DENOMINADO '' CRUZADA EVANGELITICA '' DEPARAMOS COM O AUTOR DISCUTINDO COM O SEU CUNHADO CRISTIAM. QUE A VITIMA E UM DOS ORGANIZADORES DO EVENTO, ONDE AO AVISTAR A DISCUSSAO TENTOU RESOLVER O PROBLEMA, MOMENTO QUE O AUTOR POR MOTIVOS DESCONHECIDOS DESFERIU UM SOCO CONTRA O ROSTO DA MESMA O QUE RESULTOU EM UM HEMATOMA NO LADO ESQUERDO DA FACE. CRISTIAM NAO FOI ENCONTRADO NO LOCAL. DO EXPOSTO O AUTOR FOI PRESO EM FLAGRANTE DELITO E ENCAMINHADO EM PERFEITAS CONDICOES FISICAS PARA ESTA DEPOL. A VITIMA E AUTOR FORAM ENCAMINHADOS AO PAM ONDE FORAM ATENDIDOS PELO MEDICO PLANTONISTA. SEGUE ANEXO 02 RELATORIOS MEDICO. FOI FEITO USO DE ALGEMAS PARA SEGURANCA DO AUTOR E DA EQUIPE POLICIAL.   "]

In [14]:
df.codigo.value_counts()

B01129    290000
T00008    290000
C01157    290000
C01171    290000
C01155    290000
T00009    290000
Name: codigo, dtype: int64

In [15]:
df.isnull().sum()

des_historico    665
codigo             0
dtype: int64

In [16]:
df.dropna(subset=['des_historico'], inplace=True)

In [12]:
df.replace(to_replace=r'\n', value=' ', regex=True,inplace=True)
df.replace(to_replace=r'\t', value=' ', regex=True,inplace=True)
df.replace(to_replace=r'•', value='', regex=True,inplace=True)

In [17]:
text = ' '.join(df['des_historico'].values)

In [18]:
print('Total de Caracteres no Corpus:', len(text))
chars = sorted(list(set(text)))
print('Total de Caracteres Únicos:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Total de Caracteres no Corpus: 1785234823
Total de Caracteres Únicos: 197


In [14]:
print('Total de Caracteres no Corpus:', len(text))
chars = sorted(list(set(text)))
print('Total de Caracteres Únicos:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Total de Caracteres no Corpus: 79650970
Total de Caracteres Únicos: 158


In [19]:
df['codigo'].value_counts().sort_values(ascending=False).iplot(kind='bar', yTitle='Número de textos em cada ocorrência', 
                                                                title='Número de ocorrências de um total de 1.740.000')

In [20]:
def print_plot(index):
    example = df[df.index == index][['des_historico', 'codigo']].values[0]
    if len(example) > 0:
        print(example[0])
        print('codigo:', example[1])

In [21]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
# STOPWORDS = set(stopwords.words('portuguese'))
STOPWORDS = set(open('stopwords.txt','r').read().split('\n'))

In [22]:
def clean_text(text):
    text = text.lower() # lowercase text
    text = unidecode.unidecode(text)
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

In [23]:
df['des_historico'] = df['des_historico'].apply(clean_text)

In [25]:
# carrega os dados
#df = pd.read_csv('./Data/Ocorrencias_290.csv', encoding='utf8')
#df

,Unnamed: 0,des_historico,codigo
0,0,durante patrulhamento acionados vitima jeane r...,B01129
1,1,deslocamos ate rua jose lindolfo leite 412 cen...,B01129
2,2,acionados comparecemos local supra citado onde...,B01129
3,3,patrulhamento local guarnicao deparou vitima q...,B01129
4,4,segundo vitima apos desentendimento familiar a...,B01129
...,...,...,...
1739330,1739995,procurados senhor clenon relatou data 02 11 20...,T00009
1739331,1739996,empenhados copom comparecemos local avenida br...,T00009
1739332,1739997,acionados copom deslocamos cruzamento avenida ...,T00009
1739333,1739998,patrulhamento deparamos vp 26995 comandada sen...,T00009


In [26]:
df.codigo.value_counts()

T00008    289971
C01155    289938
T00009    289918
C01157    289913
B01129    289903
C01171    289692
Name: codigo, dtype: int64

In [67]:
df.codigo.value_counts()

ACIDENTE DE TRANSITO SEM VITIMA    289971
FURTO                              289938
ACIDENTE DE TRANSITO COM VITIMA    289918
ROUBO                              289913
LESAO CORPORAL                     289903
ESTELIONATO                        289692
Name: codigo, dtype: int64

In [28]:
df.isnull().sum()

Unnamed: 0         0
des_historico    110
codigo             0
dtype: int64

In [29]:
df.dropna(subset=['des_historico'], inplace=True)

In [30]:
df.isnull().sum()

Unnamed: 0       0
des_historico    0
codigo           0
dtype: int64

In [31]:
df.codigo.value_counts()

T00008    289955
C01155    289917
T00009    289909
C01157    289898
B01129    289884
C01171    289662
Name: codigo, dtype: int64

In [32]:
# Salva arquivo de ocorrências
df.to_csv(r'./Data/Ocorrencias_290.csv', encoding='utf8')

In [33]:
text1 = ' '.join(df['des_historico'].values)
print('Total de Caracteres no Corpus:', len(text1))
chars1 = sorted(list(set(text1)))
print('Total de Caracteres Únicos:', len(chars1))
char_indices1 = dict((c, i) for i, c in enumerate(chars1))
indices_char1 = dict((i, c) for i, c in enumerate(chars1))

Total de Caracteres no Corpus: 1365688533
Total de Caracteres Únicos: 40


In [38]:
text1 = ' '.join(df['des_historico'].values)
print('Total de Caracteres no Corpus:', len(text1))
chars1 = sorted(list(set(text1)))
print('Total de Caracteres Únicos:', len(chars1))
char_indices1 = dict((c, i) for i, c in enumerate(chars1))
indices_char1 = dict((i, c) for i, c in enumerate(chars1))

Total de Caracteres no Corpus: 61087692
Total de Caracteres Únicos: 40


In [20]:
text1 = ' '.join(df['des_historico'].values)
print('Total de Caracteres no Corpus:', len(text1))
chars1 = sorted(list(set(text1)))
print('Total de Caracteres Únicos:', len(chars1))
char_indices1 = dict((c, i) for i, c in enumerate(chars1))
indices_char1 = dict((i, c) for i, c in enumerate(chars1))

Total de Caracteres no Corpus: 11548824
Total de Caracteres Únicos: 40


In [34]:
# original
reg19

["DURANTE REALIZACAO DE POLICIAMENTO NO LOCAL DENOMINADO '' CRUZADA EVANGELITICA '' DEPARAMOS COM O AUTOR DISCUTINDO COM O SEU CUNHADO CRISTIAM. QUE A VITIMA E UM DOS ORGANIZADORES DO EVENTO, ONDE AO AVISTAR A DISCUSSAO TENTOU RESOLVER O PROBLEMA, MOMENTO QUE O AUTOR POR MOTIVOS DESCONHECIDOS DESFERIU UM SOCO CONTRA O ROSTO DA MESMA O QUE RESULTOU EM UM HEMATOMA NO LADO ESQUERDO DA FACE. CRISTIAM NAO FOI ENCONTRADO NO LOCAL. DO EXPOSTO O AUTOR FOI PRESO EM FLAGRANTE DELITO E ENCAMINHADO EM PERFEITAS CONDICOES FISICAS PARA ESTA DEPOL. A VITIMA E AUTOR FORAM ENCAMINHADOS AO PAM ONDE FORAM ATENDIDOS PELO MEDICO PLANTONISTA. SEGUE ANEXO 02 RELATORIOS MEDICO. FOI FEITO USO DE ALGEMAS PARA SEGURANCA DO AUTOR E DA EQUIPE POLICIAL.   "]

In [35]:
print_plot(19)

durante realizacao policiamento local denominado cruzada evangelitica deparamos autor discutindo cunhado cristiam vitima organizadores evento onde avistar discussao tentou resolver problema momento autor motivos desconhecidos desferiu soco contra rosto mesma resultou hematoma lado esquerdo face cristiam nao encontrado local exposto autor preso flagrante delito encaminhado perfeitas condicoes fisicas depol vitima autor encaminhados pam onde atendidos medico plantonista segue anexo 02 relatorios medico feito uso algemas seguranca autor equipe policial
codigo: B01129


# Fim